In [1]:
train = pd.read_csv('datasets/titanic/train.csv')

In [3]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [13]:
# 丢弃了姓名、客舱 Cabin 、船票信息 Ticket
train.drop(columns=['Name', 'Cabin', 'Ticket'], inplace=True)

In [17]:
train['Sex'] = train['Sex'].map(lambda x: 1 if x == 'male' else 0)

In [27]:
# Embarked 登船港口
labels = train['Embarked'].unique().tolist()
labels

['S', 'C', 'Q', nan]

In [32]:
labels.index('S'), labels.index('C'), labels.index('Q'), labels.index(np.nan)

(0, 1, 2, 3)

In [50]:
train['Embarked'] = train['Embarked'].apply(lambda n: labels.index(n))

In [55]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Embarked       891 non-null int64
dtypes: float64(2), int64(7)
memory usage: 62.7 KB


In [60]:
# 处理缺失数据，有点简单粗暴
train.fillna(0, inplace=True)

In [59]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Embarked       891 non-null int64
dtypes: float64(2), int64(7)
memory usage: 62.7 KB


In [74]:
y = train['Survived'].values
X = train.drop(columns=['Survived']).values

In [75]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=666)

In [82]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

In [83]:
train_score

1.0

In [84]:
test_score

0.6927374301675978

明显是过拟合了，用随机森林试试看。

In [97]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=500, max_depth=10)

In [98]:
rfr.fit(X_train, y_train)
train_score = rfr.score(X_train, y_train)
test_score = rfr.score(X_test, y_test)

In [99]:
train_score

0.8702282890218003

In [100]:
test_score

0.39633112821027966

In [116]:
X_train.shape

(712, 8)

In [118]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()


parameters = {'max_depth': list(range(2, 10)),
              'min_samples_split': list(range(2, 10)),
              'min_samples_leaf': list(range(2, 10)),
              'max_features': list(range(1, 9))}

clf = GridSearchCV(dt, parameters, cv=5)
clf.fit(X_train, y_train)

In [121]:
clf.best_params_

{'max_depth': 7,
 'max_features': 4,
 'min_samples_leaf': 7,
 'min_samples_split': 8}

In [122]:
clf.best_estimator_.score(X_test, y_test)

0.7597765363128491

In [125]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=500)


parameters = {'max_depth': list(range(2, 10)),
              'min_samples_split': list(range(2, 10)),
              'min_samples_leaf': list(range(2, 10)),
              'max_features': list(range(1, 9))}

clf = GridSearchCV(rfr, parameters, cv=5,verbose=2)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 4096 candidates, totalling 20480 fits
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=3 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=3, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=3 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=3, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=3 
[CV]  max_

[CV]  max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=4, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=4 
[CV]  max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=4, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=4 
[CV]  max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=4, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=4 
[CV]  max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=4, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=5 
[CV]  max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=5, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=5 
[CV]  max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=5, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=3, min_samples_split=5 
[CV]  max_

[CV]  max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=6, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=6 
[CV]  max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=6, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=6 
[CV]  max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=6, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=6 
[CV]  max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=6, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=7 
[CV]  max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=7, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=7 
[CV]  max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=7, total=   0.5s
[CV] max_depth=2, max_features=1, min_samples_leaf=4, min_samples_split=7 
[CV]  max_

KeyboardInterrupt: 

In [126]:
from sklearn.model_selection import GridSearchCV

entropy_thresholds = np.linspace(0, 1, 50)
gini_thresholds = np.linspace(0, 0.5, 50)

# Set the parameters by cross-validation
param_grid = [{'criterion': ['entropy'],
               'min_impurity_split': entropy_thresholds},
              {'criterion': ['gini'],
               'min_impurity_split': gini_thresholds},
              {'max_depth': range(2, 10)},
              {'min_samples_split': range(2, 30, 2)}]

clf = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, verbose=2)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 122 candidates, totalling 610 fits
[CV] criterion=entropy, min_impurity_split=0.0 .......................
[CV] ........ criterion=entropy, min_impurity_split=0.0, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.0 .......................
[CV] ........ criterion=entropy, min_impurity_split=0.0, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.0 .......................
[CV] ........ criterion=entropy, min_impurity_split=0.0, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.0 .......................
[CV] ........ criterion=entropy, min_impurity_split=0.0, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.0 .......................
[CV] ........ criterion=entropy, min_impurity_split=0.0, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.02040816326530612 .......
[CV]  criterion=entropy, min_impurity_split=0.02040816326530612, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.02040816326530612 .......
[CV]  

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: Deprecation

[CV]  criterion=entropy, min_impurity_split=0.2040816326530612, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.2040816326530612 ........
[CV]  criterion=entropy, min_impurity_split=0.2040816326530612, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.22448979591836732 .......
[CV]  criterion=entropy, min_impurity_split=0.22448979591836732, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.22448979591836732 .......
[CV]  criterion=entropy, min_impurity_split=0.22448979591836732, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.22448979591836732 .......
[CV]  criterion=entropy, min_impurity_split=0.22448979591836732, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.22448979591836732 .......
[CV]  criterion=entropy, min_impurity_split=0.22448979591836732, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.22448979591836732 .......
[CV]  criterion=entropy, min_impurity_split=0.22448979591836732, total=   0.0s
[CV] criterion=entropy,

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in v

[CV]  criterion=entropy, min_impurity_split=0.44897959183673464, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.44897959183673464 .......
[CV]  criterion=entropy, min_impurity_split=0.44897959183673464, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.44897959183673464 .......
[CV]  criterion=entropy, min_impurity_split=0.44897959183673464, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.44897959183673464 .......
[CV]  criterion=entropy, min_impurity_split=0.44897959183673464, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.44897959183673464 .......
[CV]  criterion=entropy, min_impurity_split=0.44897959183673464, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.4693877551020408 ........
[CV]  criterion=entropy, min_impurity_split=0.4693877551020408, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.4693877551020408 ........
[CV]  criterion=entropy, min_impurity_split=0.4693877551020408, total=   0.0s
[CV] criterion=entropy,

...

[CV] ........... criterion=gini, min_impurity_split=0.5, total=   0.0s
[CV] criterion=gini, min_impurity_split=0.5 ..........................
[CV] ........... criterion=gini, min_impurity_split=0.5, total=   0.0s
[CV] criterion=gini, min_impurity_split=0.5 ..........................
[CV] ........... criterion=gini, min_impurity_split=0.5, total=   0.0s
[CV] criterion=gini, min_impurity_split=0.5 ..........................
[CV] ........... criterion=gini, min_impurity_split=0.5, total=   0.0s
[CV] max_depth=2 .....................................................
[CV] ...................................... max_depth=2, total=   0.0s
[CV] max_depth=2 .....................................................
[CV] ...................................... max_depth=2, total=   0.0s
[CV] max_depth=2 .....................................................
[CV] ...................................... max_depth=2, total=   0.0s
[CV] max_depth=2 .....................................................
[CV] .

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


[CV] ............................. min_samples_split=14, total=   0.0s
[CV] min_samples_split=14 ............................................
[CV] ............................. min_samples_split=14, total=   0.0s
[CV] min_samples_split=14 ............................................
[CV] ............................. min_samples_split=14, total=   0.0s
[CV] min_samples_split=14 ............................................
[CV] ............................. min_samples_split=14, total=   0.0s
[CV] min_samples_split=16 ............................................
[CV] ............................. min_samples_split=16, total=   0.0s
[CV] min_samples_split=16 ............................................
[CV] ............................. min_samples_split=16, total=   0.0s
[CV] min_samples_split=16 ............................................
[CV] ............................. min_samples_split=16, total=   0.0s
[CV] min_samples_split=16 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done 610 out of 610 | elapsed:    1.7s finished
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'criterion': ['entropy'], 'min_impurity_split': array([0.     , 0.02041, 0.04082, 0.06122, 0.08163, 0.10204, 0.12245,
       0.14286, 0.16327, 0.18367, 0.20408, 0.22449, 0.2449 , 0.26531,
       0.28571, 0.30612, 0.32653, 0.34694, 0.36735, 0.38776, 0.40816,
       0.42857, 0.44898, 0.46...59, 0.4898 ,
       0.5    ])}, {'max_depth': range(2, 10)}, {'min_samples_split': range(2, 30, 2)}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [127]:
clf.best_params_

{'criterion': 'entropy', 'min_impurity_split': 0.7142857142857142}

In [128]:
entropy_thresholds

array([0.        , 0.02040816, 0.04081633, 0.06122449, 0.08163265,
       0.10204082, 0.12244898, 0.14285714, 0.16326531, 0.18367347,
       0.20408163, 0.2244898 , 0.24489796, 0.26530612, 0.28571429,
       0.30612245, 0.32653061, 0.34693878, 0.36734694, 0.3877551 ,
       0.40816327, 0.42857143, 0.44897959, 0.46938776, 0.48979592,
       0.51020408, 0.53061224, 0.55102041, 0.57142857, 0.59183673,
       0.6122449 , 0.63265306, 0.65306122, 0.67346939, 0.69387755,
       0.71428571, 0.73469388, 0.75510204, 0.7755102 , 0.79591837,
       0.81632653, 0.83673469, 0.85714286, 0.87755102, 0.89795918,
       0.91836735, 0.93877551, 0.95918367, 0.97959184, 1.        ])

In [129]:
clf.best_estimator_.score(X_test, y_test)

0.7877094972067039